### Resample all sentinel-2 images in a shapefile

Organize files by month

In [5]:
ifolder = '/mnt/c/s2_coimbra21'

ofolder = '/mnt/c/s2_vflores21'

refrst = '/mnt/c/stdrst_valeflores.tif'

In [2]:
import pandas as pd
import os
from glass.rd.shp import shp_to_obj
from glass.pys.tm import now_as_str
from glass.pys.oss import lst_ff, mkdir, del_folder
from glass.dtt.stl  import unzip_img
from glass.rst.sat.rmp  import resample_s2img

In [10]:
imgs = lst_ff(ifolder, file_format='.zip', rfilename=True)

In [12]:
_imgs = [[i] + i[:-4].split('_') for i in imgs]

In [18]:
idf = pd.DataFrame(_imgs, columns=[
    'filename', 'satelite', 'sensor',
    'sensortime', 'n', 'r',
    'cellid', 'prodtime'
])

idf['sensortime'] = pd.to_datetime(idf.sensortime, format='%Y%m%dT%H%M%S')
idf['sensortime'] = idf.sensortime.dt.floor('s')
idf['sensormonth'] = idf.sensortime.dt.month
idf['prodtime'] = pd.to_datetime(idf.prodtime, format='%Y%m%dT%H%M%S')

In [19]:
months = idf.sensormonth.unique()
months.sort()

In [20]:
months

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int32)

In [17]:
idf

,filename,satelite,sensor,sensortime,n,r,cellid,prodtime
0,S2A_MSIL2A_20210517T113321_N0300_R080_T29TNE_2...,S2A,MSIL2A,2021-05-17 11:33:21,N0300,R080,T29TNE,20210517T144952
1,S2B_MSIL2A_20210228T112109_N0214_R037_T29TNE_2...,S2B,MSIL2A,2021-02-28 11:21:09,N0214,R037,T29TNE,20210228T141025
2,S2B_MSIL2A_20210728T112119_N0301_R037_T29TNE_2...,S2B,MSIL2A,2021-07-28 11:21:19,N0301,R037,T29TNE,20210728T143728
3,S2A_MSIL2A_20211223T113511_N0301_R080_T29TNE_2...,S2A,MSIL2A,2021-12-23 11:35:11,N0301,R080,T29TNE,20211223T142742
4,S2A_MSIL2A_20210812T112121_N0301_R037_T29TNE_2...,S2A,MSIL2A,2021-08-12 11:21:21,N0301,R037,T29TNE,20210812T143918
...,...,...,...,...,...,...,...,...
140,S2A_MSIL2A_20210616T113321_N0300_R080_T29TNE_2...,S2A,MSIL2A,2021-06-16 11:33:21,N0300,R080,T29TNE,20210616T174356
141,S2B_MSIL2A_20210323T113319_N0214_R080_T29TNE_2...,S2B,MSIL2A,2021-03-23 11:33:19,N0214,R080,T29TNE,20210323T130220
142,S2A_MSIL2A_20211014T113321_N0301_R080_T29TNE_2...,S2A,MSIL2A,2021-10-14 11:33:21,N0301,R080,T29TNE,20211014T143321
143,S2A_MSIL2A_20211130T112431_N0301_R037_T29TNE_2...,S2A,MSIL2A,2021-11-30 11:24:31,N0301,R037,T29TNE,20211130T142403


In [ ]:
for m in months:
    imgs = idf[idf.month == m]
    
    if not imgs.shape[0]: continue
    
    mf = mkdir(os.path.join(ofolder, f"m_{str(m)}"))
    
    for i, r in imgs.iterrows():
        resample_s2img(
            os.path.join(ifolder, r['filename']),
            refrst,
            ofolder,
            bands=['b02', 'b03', 'b04', 'b05', 'b06', 'b07', 'b08', 'b11', 'b12']
        )